In [2]:
import random
import time
import numpy as np
import pandas as pd

### 🔧 Data Generator

Fungsi `generate_data(n_murid, n_tutor)` digunakan untuk membuat dataset simulasi matching antara murid dan tutor.

- Setiap murid diwakili dengan ID integer.
- Setiap tutor memiliki kapasitas (1–3 murid).
- Preferensi antara murid-tutor dinilai berdasarkan 4 aspek:
  - Kecocokan waktu
  - Kecocokan topik
  - Mode belajar
  - Gaya belajar

Preferensi ini dinyatakan dengan nilai 0 atau 1 dan disimpan dalam dictionary berpasangan (murid, tutor).


In [4]:
def generate_data(n_murid, n_tutor, seed=42):
    random.seed(seed)
    murid = list(range(n_murid))
    tutor = list(range(n_tutor))
    kapasitas = {t: random.randint(1, 3) for t in tutor}
    preferensi = {
        (m, t): {
            "waktu": random.randint(0, 1),
            "topik": random.randint(0, 1),
            "mode": random.randint(0, 1),
            "gaya": random.randint(0, 1),
        }
        for m in murid for t in tutor
    }
    return murid, tutor, kapasitas, preferensi

### 🧩 CSP Matching (Sederhana)

Fungsi `csp_matching()` mencoba membuat assignment satu-per-satu berdasarkan sisa kapasitas tutor.

- Untuk setiap murid, fungsi akan memilih tutor pertama yang memiliki kapasitas tersisa.
- Tidak memperhitungkan skor preferensi, fokus pada constraint kapasitas.

Kelebihan:
- Sangat cepat (greedy).
- Solusi valid terhadap constraint kapasitas.

Kekurangan:
- Tidak optimal dalam hal kualitas preferensi (skor total rendah).


In [5]:
def csp_matching(murid, tutor, kapasitas, preferensi):
    assignment = {}
    sisa_kapasitas = kapasitas.copy()
    for m in murid:
        for t in tutor:
            if sisa_kapasitas[t] > 0:
                assignment[m] = t
                sisa_kapasitas[t] -= 1
                break
    return assignment

### 🔥 Simulated Annealing (SA) Matching

Fungsi `sa_matching()` menerapkan algoritma **Simulated Annealing** untuk mengeksplorasi solusi matching dengan skor preferensi setinggi mungkin.

- Inisialisasi dengan solusi acak.
- Pada setiap iterasi:
  - Pilih satu murid secara acak dan coba tukar tutornya.
  - Hitung perubahan skor total (`Δ`).
  - Jika `Δ > 0`, terima perubahan.
  - Jika `Δ < 0`, terima dengan probabilitas `p = exp(Δ / T)`.
- Temperatur `T` menurun perlahan (cooling schedule) untuk mengurangi eksplorasi buruk seiring waktu.

SA memungkinkan untuk:
- Mencapai solusi berkualitas tinggi.
- Keluar dari jebakan lokal dengan mekanisme probabilistik.


In [6]:
def sa_matching(murid, tutor, kapasitas, preferensi, T0=1.0, cooling=0.99, steps=1000):
    current = {m: random.choice(tutor) for m in murid}
    best = current.copy()

    def score(assign):
        return sum(sum(preferensi[(m, t)].values()) for m, t in assign.items())

    best_score = score(current)
    T = T0

    for _ in range(steps):
        new = current.copy()
        m = random.choice(murid)
        new[m] = random.choice(tutor)
        delta = score(new) - score(current)
        if delta > 0 or random.random() < np.exp(delta / T):
            current = new
            if score(current) > best_score:
                best = current
                best_score = score(current)
        T *= cooling

    return best

### 📊 Evaluasi Kualitas Solusi dan Constraint

Fungsi `evaluate()` menghitung dua metrik utama untuk setiap hasil matching:

1. **Skor Total**: jumlah poin dari seluruh aspek preferensi yang cocok.
2. **Persentase Constraint Terpenuhi**: dihitung dari (jumlah atribut cocok) dibagi total maksimum (4 aspek × jumlah murid).

Ini membantu menilai:
- Seberapa baik kualitas solusi (tingginya skor).
- Seberapa banyak preferensi pengguna yang berhasil dipenuhi.


In [7]:
def evaluate(matching, preferensi):
    total_score = sum(sum(preferensi[(m, t)].values()) for m, t in matching.items())
    total_constraint = 4 * len(matching)
    terpenuhi = total_score
    persentase = terpenuhi / total_constraint * 100
    return total_score, persentase

### 🚀 Eksperimen CSP vs SA Matching

Fungsi `run_test()` menjalankan evaluasi pada tiga ukuran data berbeda: 5, 10, dan 20 pasangan.

- Untuk setiap ukuran:
  - Generate data
  - Jalankan CSP Matching → ukur waktu dan skor
  - Jalankan SA Matching → ukur waktu dan skor
- Hasil disimpan dalam DataFrame.

Metrik yang dibandingkan:
- Skor total (kualitas solusi)
- % constraint terpenuhi (preferensi)
- Waktu eksekusi (perf_counter)


In [8]:
def run_test():
    results = []
    for size in [5, 10, 20]:
        murid, tutor, kapasitas, preferensi = generate_data(size, size)

        # CSP
        t0 = time.perf_counter()
        csp_match = csp_matching(murid, tutor, kapasitas, preferensi)
        t1 = time.perf_counter()
        csp_score, csp_constraint = evaluate(csp_match, preferensi)

        # SA
        t2 = time.perf_counter()
        sa_match = sa_matching(murid, tutor, kapasitas, preferensi)
        t3 = time.perf_counter()
        sa_score, sa_constraint = evaluate(sa_match, preferensi)

        results.append({
            "Ukuran": size,
            "CSP_Skor": csp_score,
            "CSP_%Constraint": round(csp_constraint, 2),
            "CSP_Waktu (s)": round(t1 - t0, 6),
            "SA_Skor": sa_score,
            "SA_%Constraint": round(sa_constraint, 2),
            "SA_Waktu (s)": round(t3 - t2, 6)
        })

    return pd.DataFrame(results)

### 📈 Hasil Akhir dan Kesimpulan

Dengan menjalankan `run_test()`, kita bisa membandingkan efektivitas dan efisiensi CSP vs SA untuk berbagai ukuran dataset.

📌 Gunakan tabel hasil untuk menganalisis:
- Mana algoritma yang memberikan **skor lebih tinggi**?
- Mana yang lebih cepat?
- Seberapa besar preferensi user bisa dipenuhi oleh tiap algoritma?

Visualisasi tambahan seperti grafik waktu dan kualitas bisa ditambahkan untuk memperkuat analisis.


In [9]:
df = run_test()
print("\nHasil Evaluasi CSP vs Simulated Annealing:\n")
print(df.to_string(index=False))


Hasil Evaluasi CSP vs Simulated Annealing:

 Ukuran  CSP_Skor  CSP_%Constraint  CSP_Waktu (s)  SA_Skor  SA_%Constraint  SA_Waktu (s)
      5         5             25.0       0.000006       15            75.0      0.008956
     10        18             45.0       0.000006       33            82.5      0.011274
     20        38             47.5       0.000012       74            92.5      0.017010


## 📊 Analisis Hasil Evaluasi CSP vs Simulated Annealing

### 📌 Tujuan Evaluasi
Membandingkan performa dua algoritma — **CSP (Constraint Satisfaction Problem)** dan **Simulated Annealing (SA)** — dalam menyelesaikan masalah matching 1-on-1 (tutor-murid), berdasarkan:

1. **Kualitas Solusi** (Skor preferensi total)
2. **Pemenuhan Constraint** (% atribut preferensi yang cocok)
3. **Waktu Eksekusi**
4. **Skalabilitas saat ukuran data bertambah**

---

### 📈 Hasil Pengamatan

| Ukuran | CSP_Skor | CSP_%Constraint | CSP_Waktu (s) | SA_Skor | SA_%Constraint | SA_Waktu (s) |
|--------|----------|------------------|----------------|---------|----------------|----------------|
|   5    |    5     |      25.0%       |    0.000006    |   15    |     75.0%      |    0.008956    |
|  10    |   18     |      45.0%       |    0.000006    |   33    |     82.5%      |    0.011274    |
|  20    |   38     |      47.5%       |    0.000012    |   74    |     92.5%      |    0.017010    |

---

### 🔍 Analisis:

#### ✅ **Kualitas Solusi (Skor)**
- SA menghasilkan skor **jauh lebih tinggi** dibanding CSP di semua ukuran.
- Skor SA meningkat lebih tajam saat ukuran bertambah, menunjukkan eksplorasi solusi lebih optimal.

#### ✅ **Pemenuhan Constraint**
- CSP hanya memenuhi sekitar 25%–47.5% preferensi.
- SA mencapai hingga **92.5% pemenuhan constraint** pada ukuran 20, menunjukkan adaptabilitas terhadap preferensi.

#### ✅ **Waktu Eksekusi**
- CSP sangat cepat (< 20µs) karena bersifat greedy dan deterministik.
- SA sedikit lebih lambat (hingga 17ms), namun **masih tergolong sangat cepat** untuk ukuran kecil-menengah.

#### ✅ **Skalabilitas**
- SA menunjukkan **kualitas solusi meningkat** seiring ukuran.
- CSP stagnan bahkan saat ukuran meningkat → tidak eksploratif.

---

### 🏁 Kesimpulan:

- **CSP cocok untuk solusi cepat dan valid minimal**, namun kualitas solusinya rendah.
- **Simulated Annealing unggul dari segi skor, fleksibilitas, dan pemenuhan preferensi**, dengan sedikit trade-off di runtime.
- Untuk matching dengan banyak preferensi/atribut, **SA lebih disarankan** terutama saat kualitas lebih penting daripada kecepatan absolut.

